# Mastra
### Downloading the mastra dataset
*Written by - Rasmus Bergman rbvp20@student.aau.dk*

This notebook for downloading the open dataset from Mastra

In [2]:
# Import Libraries
import urllib.request
import pyproj
import json
import geopandas as gpd
import os
import constants as c

# Constants
X_SLICES = 10
Y_SLICES = 15

In [71]:
# The api is restricted to a maximum of 3000 data points.
# Therefore it is necessary to get the data in smaller areas.
# The bounding box of Denmark is used to divide the area into smaller parts.
# This bounding box is then sliced into 10x15 parts.
# The data is then fetched from the api in these smaller areas.

DK_BBOX = (8.08997684086, 54.8000145534, 12.6900061378, 57.730016588)
source_crs = pyproj.CRS("EPSG:4326")
target_crs = pyproj.CRS("EPSG:25832")
string_encoding = 'latin-1' 
# The data is encoded in latin-1 maybe?
# For 1 of the files downloaded it is not encoded in latin-1, so it must
# be fetched manually and saved in the "data/mastra" folder with the name
# specified in the printout.

transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

mastra_url_start = "https://vmgeoserver.vd.dk/geosmastra/opendata/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=OPEN_DATA_NOEGLETAL_VIEW&outputFormat=json&encoding=utf-8"
def construct_mastra_url(lon_start: float, lon_end: float, lat_start: float, lat_end: float):
    return mastra_url_start+f"&CQL_FILTER=BBOX(KOOR_SDO,{lon_start},{lat_start},{lon_end},{lat_end})"

for x in X_SLICES:
    for y in Y_SLICES:
        lon_start = DK_BBOX[0] + (DK_BBOX[2] - DK_BBOX[0]) / X_SLICES * x
        lon_end = DK_BBOX[0] + (DK_BBOX[2] - DK_BBOX[0]) / X_SLICES * (x+1)
        lat_start = DK_BBOX[1] + (DK_BBOX[3] - DK_BBOX[1]) / Y_SLICES * y
        lat_end = DK_BBOX[1] + (DK_BBOX[3] - DK_BBOX[1]) / Y_SLICES * (y+1)

        lon_start, lat_start = transformer.transform(lon_start, lat_start)
        lon_end, lat_end = transformer.transform(lon_end, lat_end)

        url = construct_mastra_url(lon_start, lon_end, lat_start, lat_end)
        result = urllib.request.urlopen(url).read()

        file_path = os.path.join(c.MASTRA_FOLDER, f"{x}_{y}.json")

        # Try catch block for the encoding error
        try:
            decoded = result.decode(string_encoding)
            if '"numberReturned":0' in decoded:
                continue
            with open(file_path, 'w') as f:
                f.write(decoded)
        except UnicodeEncodeError as e:
            print(f"Error at: ({x}_{y}), go to the following url and save the file as {x}_{y}.json")
            print(url)

Error writing to ../data/mastra/1_11.json
https://vmgeoserver.vd.dk/geosmastra/opendata/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=OPEN_DATA_NOEGLETAL_VIEW&outputFormat=json&encoding=utf-8&CQL_FILTER=BBOX(KOOR_SDO,472624.7229623238,6311763.654711639,500604.0833842272,6333417.359266506)


In [ ]:
years = dict()
unique_features = []

for file in os.listdir(c.MASTRA_DIR):
    with open(os.path.join(c.MASTRA_DIR, file), 'r') as f:
        features = json.load(f)["features"]
        for feature in features:
            road_id = str(feature["properties"]["VEJNR"]) + "-" + feature["properties"]["VEJDEL"]
            if road_id in years:
                old_year = years[road_id]
                new_year = feature["properties"]["AAR"]
                if new_year < old_year:
                    continue
            years[road_id] = feature["properties"]["AAR"]
            unique_features.append(feature)

gdf = gpd.GeoDataFrame.from_features(unique_features, "EPSG:25832")
gdf.to_crs("EPSG:4326", inplace=True)

gdf.to_file(c.MASTRA_PATH, driver="GeoJSON")